In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk

# Download the VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

# Initialize VADER Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

# Sample data
concerns = [
    "I'm feeling really anxious and can't sleep.",
    "I am just a bit stressed.",
    "I am extremely depressed and overwhelmed with everything."
]

# Feature extraction function
def extract_features(text):
    # VADER sentiment score (compound value)
    sentiment_score = sia.polarity_scores(text)["compound"]
    
    # Basic keyword analysis (count of emotional keywords)
    keywords = ["anxiety", "depression", "stress", "overwhelmed", "panic"]
    keyword_count = sum(1 for word in keywords if word in text.lower())
    
    # Polarity Shift (additional placeholder, assuming tracking over time)
    # Here, assuming a static polarity shift of 0.1 for demonstration purposes
    polarity_shift = 0.1  # Adjust based on previous user inputs in a real scenario
    
    # Return as numpy array
    return np.array([sentiment_score, keyword_count, polarity_shift])

# Create feature and target arrays
X = np.array([extract_features(text) for text in concerns])
y = np.array([8, 4, 9])  # Sample intensity scores (1-10), ideally provided as labeled data

# Scale features for neural network input
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)

# Define a simple neural network model
class IntensityScoreModel(nn.Module):
    def _init_(self):
        super(IntensityScoreModel, self)._init_()
        self.fc1 = nn.Linear(3, 10)  # Input layer with 3 features
        self.fc2 = nn.Linear(10, 1)  # Output layer with a single intensity score
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = IntensityScoreModel()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X).squeeze()
    loss = criterion(outputs, y)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing model prediction on new concerns
new_concern = "I feel mildly anxious but managing it."
new_features = extract_features(new_concern)
new_features = scaler.transform(new_features.reshape(1, -1))
new_features = torch.FloatTensor(new_features)
predicted_intensity = model(new_features).item()

print(f'Predicted Intensity Score for new concern: {predicted_intensity:.2f}')